In [20]:
#import csv
#import random
#import pickle
#import os
#import codecs
#import matplotlib.pyplot as plt
#import numpy as np
#from urllib.request import urlopen
#from deap import base, creator, tools, algorithms
#
#from Route import TravelingSalesmanProblem

In [26]:
import csv
import pickle
import os
import codecs

import numpy as np

from urllib.request import urlopen

import matplotlib.pyplot as plt

class TravelingSalesmanProblem:

    def __init__(self, name):
     
        # initialize instance variables:
        self.name = name
        self.locations = []
        self.distances = []
        self.tspSize = 0

        # initialize the data:
        self.__initData()

    def __len__(self):
        return self.tspSize

    def __initData(self):


        # attempt to read serialized data:
        try:
            self.locations = pickle.load(open(os.path.join("tsp-data", self.name + "-loc.pickle"), "rb"))
            self.distances = pickle.load(open(os.path.join("tsp-data", self.name + "-dist.pickle"), "rb"))
        except (OSError, IOError):
            pass

        # serailized data not found - create the data from scratch:
        if not self.locations or not self.distances:
            self.__createData()

        # set the problem 'size':
        self.tspSize = len(self.locations)

    def __createData(self):

        self.locations = []

        # open whitespace-delimited file from url and read lines from it:
        with urlopen("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/" + self.name + ".tsp") as f:
            reader = csv.reader(codecs.iterdecode(f, 'utf-8'), delimiter=" ", skipinitialspace=True)

            # skip lines until one of these lines is found:
            for row in reader:
                if row[0] in ('DISPLAY_DATA_SECTION', 'NODE_COORD_SECTION'):
                    break

            # read data lines until 'EOF' found:
            for row in reader:
                if row[0] != 'EOF':
                    # remove index at beginning of line:
                    del row[0]

                    # convert x,y coordinates to ndarray:
                    self.locations.append(np.asarray(row, dtype=np.float32))
                else:
                    break

            # set the problem 'size':
            self.tspSize = len(self.locations)

            # print data:
            print("length = {}, locations = {}".format(self.tspSize, self.locations))

            # initialize distance matrix by filling it with 0's:
            self.distances = [[0] * self.tspSize for _ in range(self.tspSize)]

            # populate the distance matrix with calculated distances:
            for i in range(self.tspSize):
                for j in range(i + 1, self.tspSize):
                    # calculate euclidean distance between two ndarrays:
                    distance = np.linalg.norm(self.locations[j] - self.locations[i])
                    self.distances[i][j] = distance
                    self.distances[j][i] = distance
                    print("{}, {}: location1 = {}, location2 = {} => distance = {}".format(i, j, self.locations[i], self.locations[j], distance))

            # serialize locations and distances:
            if not os.path.exists("tsp-data"):
                os.makedirs("tsp-data")
            pickle.dump(self.locations, open(os.path.join("tsp-data", self.name + "-loc.pickle"), "wb"))
            pickle.dump(self.distances, open(os.path.join("tsp-data", self.name + "-dist.pickle"), "wb"))

    def getTotalDistance(self, indices):

        # distance between th elast and first city:
        distance = self.distances[indices[-1]][indices[0]]

        # add the distance between each pair of consequtive cities:
        for i in range(len(indices) - 1):
            distance += self.distances[indices[i]][indices[i + 1]]

        return distance

    def plotData(self, indices):

        # plot the dots representing the cities:
        plt.scatter(*zip(*self.locations), marker='.', color='red')

        # create a list of the corresponding city locations:
        locs = [self.locations[i] for i in indices]
        locs.append(locs[0])

        # plot a line between each pair of consequtive cities:
        plt.plot(*zip(*locs), linestyle='-', color='blue')

        return plt
    

def main():
    # create a problem instance:
    tsp = TravelingSalesmanProblem("bayg29")

    # generate a random solution and evaluate it:
    randomSolution = random.sample(range(len(tsp)), len(tsp))

    # see http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/bayg29.opt.tour
    optimalSolution = [0, 27, 5, 11, 8, 25, 2, 28, 4, 20, 1, 19, 9, 3, 14, 17, 13, 16, 21, 10, 18, 24, 6, 22, 7, 26, 15, 12, 23]

    print("Problem name: " + tsp.name)
    print("Optimal solution = ", optimalSolution)
    print("Optimal distance = ", tsp.getTotalDistance(optimalSolution))

    # plot the solution:
    plot = tsp.plotData(optimalSolution)
    plot.show()


if __name__ == "__main__":
    main()

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [21]:
LENGTH = 100
POPULATION_SIZE=100
P_CROSSOVER=0.8
P_MUTATION=0.1
MAX_GENERATIONS=50
HALL_OF_FAME_SIZE = 10

RANDOM_SEED = randint(0,100)
random.seed(RANDOM_SEED)

toolbox = base.Toolbox()

TSP_NAME = "bayg29"
tsp = TravelingSalesmanProblem(TSP_NAME)

creator.create("FitnessMin", base.Fitness, weights = (-1.0,))

creator.create("Individual", array.array, typecode = 'i', fitness = creator.FitnessMin)

toolbox.register("randomOrder", random.sample, range(len(tsp)), len(tsp))
toolbox.register("individualCreator",tools.initIterate, creator.Individual, toolbox.randomOrder)
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


NameError: name 'os' is not defined

In [19]:
def tpsDistance(individual):
    return tsp.getTotalDistance(individual),

toolbox.register("evaluate",tpsDistance)

toolbox.register("select", tools.selTournament, tournsize = 3)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb = 1.0/len(tsp))

NameError: name 'tsp' is not defined

In [ ]:
def main():
    
    population = toolbox.populationCreator(n=POPULATION_SIZE)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    
    stats.register("max", numpy.max)
    stats.register("avg", numpy.mean)
    
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
    
    population, logbook = algorithms.eaSimple(population,toolbox,
                                             cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                             ngen=MAX_GENERATIONS, halloffame= hof,
                                             stats=stats, verbose=True)
    
    maxFitnessValues, meanFitnessValues = logbook.select("max", "avg")
    
    plt.plot(maxFitnessValues, color='red')
    plt.plot(meanFitnessValues, color = "green")
    plt.xlabel('Поколение')
    plt.ylabel('Макс/средняя приспособленность')
    plt.title('Зависимость максимальной и средней приспособленности от поколения')
    plt.show()
        
    best = hof.items[0]
    tsp.plotData()